In [1]:
!pip install requests beautifulsoup4 transformers torch accelerate bitsandbytes

import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os
import sys


In [11]:
print("Setup complete. Python version:", sys.version.split()[0])
print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

# --- 1. Configuration ---
# IMPORTANT: Change this URL to the site you want to scrape!
TARGET_URL = "https://www.shaastra.org/"
OUTPUT_FILENAME = "shaastra_info.txt"
MODEL_NAME = "google/gemma-7b-it"

Setup complete. Python version: 3.12.12
PyTorch version: 2.9.0+cu126
GPU available: True


In [9]:
# ==============================================================================
# 1. Web Scraping Function (UPDATED WITH HEADERS)
# ==============================================================================

def scrape_website_content(url):
    """Fetches and extracts all text content from a given URL."""
    print(f"🌍 Starting to scrape: {url}")

    # --- FIX: disguise the scraper as a browser ---
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    try:
        # Pass the headers here
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all text content from common elements
        # Added 'div' and 'article' to catch more content on modern sites
        text_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'li', 'span', 'div', 'article'])

        # improved cleaning to remove empty lines or very short snippets
        cleaned_texts = []
        for element in text_elements:
            text = element.get_text(separator=' ', strip=True)
            if len(text) > 50: # Only keep text segments longer than 50 chars to reduce noise
                cleaned_texts.append(text)

        # Remove duplicates while preserving order
        seen = set()
        unique_text = []
        for text in cleaned_texts:
            if text not in seen:
                unique_text.append(text)
                seen.add(text)

        scraped_text = "\n\n".join(unique_text)

        if len(scraped_text) < 100:
             print("⚠️ Warning: Scraped very little text. The site might be JavaScript-heavy.")

        print(f"✅ Scraping complete. Extracted {len(scraped_text)} characters of text.")
        return scraped_text

    except requests.exceptions.RequestException as e:
        print(f"❌ An error occurred during scraping: {e}")
        return None

In [4]:
# ==============================================================================
# 2. LLM Refinement Function (Gemma-7B-IT)
# ==============================================================================

def refine_text_with_gemma(scraped_text, model_name):
    """
    Passes the scraped text to the Gemma-7B-IT model for refinement using
    4-bit quantization for memory efficiency.
    """
    if not scraped_text:
        return "No text to refine."

    # Use GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\n🧠 Initializing LLM on device: {device}")

    try:
        # Configuration for 4-bit quantization (significantly reduces VRAM usage)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for better performance
        )

        # Load the tokenizer and model with quantization
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
        )
        print("✅ Model loaded successfully using 4-bit quantization.")

        # --- Prompt Engineering ---
        prompt = (
            f"You are an expert text summarizer and cleaner. "
            f"Read the following raw, scraped text and perform two tasks:\n"
            f"1. Create a concise, professional summary of the content.\n"
            f"2. Remove all extraneous characters, poor formatting, and navigational links typical of scraped web data.\n\n"
            f"--- RAW SCRAPED TEXT (Limited to 15,000 chars for context window) ---\n"
            f"{scraped_text[:15000]}"
            f"\n\n--- END OF RAW SCRAPED TEXT ---\n"
            f"Your Refined Output (Summary and Cleaned Text):"
        )

        # Tokenize the prompt and move to the appropriate device
        input_ids = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate the response
        print("⏳ Generating refined text...")
        output = model.generate(
            **input_ids,
            max_new_tokens=2048,
            do_sample=True,
            temperature=0.6, # A bit more focused generation
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated text and clean up
        refined_text = tokenizer.decode(output[0], skip_special_tokens=True)
        # Remove the prompt part that the model echoes
        start_marker = "Your Refined Output (Summary and Cleaned Text):"
        if start_marker in refined_text:
            refined_text = refined_text.split(start_marker, 1)[-1].strip()

        print("✅ Refinement complete.")
        return refined_text

    except Exception as e:
        print(f"❌ An error occurred during LLM processing. Ensure your Colab runtime has a GPU enabled (Runtime -> Change runtime type): {e}")
        return f"LLM failed to process: {e}"


In [5]:
# ==============================================================================
# 3. Output Function
# ==============================================================================

def save_to_file(data, filename):
    """Writes the given data string to a specified file in the Colab environment."""
    try:
        # Colab file system writes to the current session's directory
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(data)

        print(f"\n💾 Successfully saved the refined output to: **{filename}**")
        print("You can download this file from the left sidebar's 'Files' icon.")

        # Optional: Display the first 500 characters of the output
        print("\n--- Start of Refined Output Preview ---")
        print(data[:500] + "..." if len(data) > 500 else data)
        print("--- End of Refined Output Preview ---")

    except IOError as e:
        print(f"❌ Could not write to file {filename}: {e}")

In [12]:
# ==============================================================================
# 4. Main Execution Block
# ==============================================================================

def main():
    """Main function to orchestrate the scraping, refining, and saving process."""

    print("\n--- Starting Web Scraper & LLM Refinement Process ---")

    # 1. Scrape the data
    scraped_content = scrape_website_content(TARGET_URL)

    if scraped_content:
        # 2. Pass to LLM for refinement
        refined_content = refine_text_with_gemma(scraped_content, MODEL_NAME)

        # 3. Save the result
        save_to_file(refined_content, OUTPUT_FILENAME)

    else:
        print("\nProcess halted due to scraping failure.")

main()


--- Starting Web Scraper & LLM Refinement Process ---
🌍 Starting to scrape: https://www.shaastra.org/
⚠️ Warning: Scraped very little text. The site might be JavaScript-heavy.
✅ Scraping complete. Extracted 0 characters of text.

Process halted due to scraping failure.
